# libraries 

In [ ]:
import requests #The requests library for HTTP requests in Python
import csv
import pandas as pd
from secrets import IEX_CLOUD_API_TOKEN_LIVE
from secrets import ALPHA_API_KEY
from secrets import API_KEY_POLYGON
import aiohttp
import asyncio
import time
from datetime import datetime, timedelta
import os 

start_time = time.time()


In [ ]:
api_url = f'https://cloud.iexapis.com/stable/ref-data/iex/symbols?format=csv&token={IEX_CLOUD_API_TOKEN_LIVE}'
symbols = []
with requests.Session() as s:
    download = s.get(api_url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list[1:]:
        symbols.append(row[0])


# function to get dates

In [ ]:
todo_dates = []

# Function to get a list of all dates that are to be downloaded
def get_dates():

    date_today = datetime.today().date()
    earliest_date = date_today - timedelta(days = 730) 

    # Get last date for which download was done to get start date
    try:
        folder_date = []
        year = max([name for name in os.listdir('output/') if not name.startswith('.')])
        foldernames = os.listdir('output/{}'.format(year))
        folder_CW = [name for name in foldernames if not name.startswith('.')]

        for i in range(len(folder_CW)):
            foldernames = os.listdir('output/{}/{}'.format(year, folder_CW[i]))
            folder_date += [name for name in foldernames if not name.startswith('.')]
        last_date = datetime.strptime(max(folder_date)[0:8], '%Y%m%d').date()

        start_date = max(earliest_date, last_date + timedelta(days=1))
    except:
        start_date = earliest_date

    # Get todo dates with weekends filtered out
    dates = [start_date + timedelta(days=i) for i in range((date_today - start_date).days)]
    todo_dates = [date.strftime('%Y%m%d') for date in dates if date.isoweekday() <= 5]

    return(todo_dates)

get_dates()


['20200127',
 '20200128',
 '20200129',
 '20200130',
 '20200131',
 '20200203',
 '20200204',
 '20200205',
 '20200206',
 '20200207',
 '20200210',
 '20200211',
 '20200212',
 '20200213',
 '20200214',
 '20200217',
 '20200218',
 '20200219',
 '20200220',
 '20200221',
 '20200224',
 '20200225',
 '20200226',
 '20200227',
 '20200228',
 '20200302',
 '20200303',
 '20200304',
 '20200305',
 '20200306',
 '20200309',
 '20200310',
 '20200311',
 '20200312',
 '20200313',
 '20200316',
 '20200317',
 '20200318',
 '20200319',
 '20200320',
 '20200323',
 '20200324',
 '20200325',
 '20200326',
 '20200327',
 '20200330',
 '20200331',
 '20200401',
 '20200402',
 '20200403',
 '20200406',
 '20200407',
 '20200408',
 '20200409',
 '20200410',
 '20200413',
 '20200414',
 '20200415',
 '20200416',
 '20200417',
 '20200420',
 '20200421',
 '20200422',
 '20200423',
 '20200424',
 '20200427',
 '20200428',
 '20200429',
 '20200430',
 '20200501',
 '20200504',
 '20200505',
 '20200506',
 '20200507',
 '20200508',
 '20200511',
 '20200512',

# Loop through the tickers and bring up the daily OHLC

In [4]:
gappedStocks = []

# difference between two numbers
def changePercent(first, second):
    first = round(first, 2)
    second = round(second, 2)
    return ((second - first) / first) * 100

async def main():

    async with aiohttp.ClientSession() as session:

        for date in todo_dates:

            for i in symbols:
                symbol = i
                
                api_url = f'https://api.polygon.io/v1/open-close/{symbol}/2020-10-14?adjusted=true&apiKey={API_KEY_POLYGON}'
                async with session.get(api_url) as resp:
                    data = await resp.json()
                    print(data)

                # for i in data:
                #     if 'Time Series (Daily)' in data:
                #         for dt, values in data['Time Series (Daily)'].items(): 

                #             # get the values from the API dictionary
                #             open = float(values['1. open'])
                #             high = float(values['2. high'])
                #             low = float(values['3. low'])
                #             close = float(values['4. close'])
                #             volume = float(values['6. volume'])
                #             gap = changePercent(close, high)
                            
                #             if gap > 15 and volume > 10000000 and open > 3 and open < 15:
                #                 gappedObj = {
                #                     'symbol': symbol,
                #                     'date': dt,
                #                     'gap (%)': gap,
                #                     'open': open,
                #                     'high': high,
                #                     'low': low,
                #                     'close': close,
                #                     'volume': volume
                #                 }
                                
                #                 gappedStocks.append(gappedObj)
                #         print(gappedStocks)


await main()
print(gappedStocks)
print("--- %s seconds ---" % (time.time() - start_time))   



{'status': 'OK', 'from': '2020-10-14', 'symbol': 'A', 'open': 105.49, 'high': 106.67, 'low': 104.92, 'close': 105.06, 'volume': 914523, 'afterHours': 105.06, 'preMarket': 105.49}
{'status': 'OK', 'from': '2020-10-14', 'symbol': 'AA', 'open': 12.79, 'high': 13.14, 'low': 12.7055, 'close': 13, 'volume': 7509107, 'afterHours': 12.38, 'preMarket': 12.7}
{'status': 'OK', 'from': '2020-10-14', 'symbol': 'AAA', 'open': 24.92, 'high': 24.92, 'low': 24.91, 'close': 24.91, 'volume': 311, 'afterHours': 24.9193, 'preMarket': 24.92}
{'status': 'OK', 'from': '2020-10-14', 'symbol': 'AAAU', 'open': 19.01, 'high': 19.05, 'low': 18.9, 'close': 18.9, 'volume': 244936, 'afterHours': 18.93, 'preMarket': 18.94}
{'status': 'NOT_FOUND', 'request_id': 'c0eac4f5544b615ac373273e993c1bf2', 'message': 'Data not found.'}
{'status': 'NOT_FOUND', 'request_id': '4430c83beedf2d61c47cc2acd7889103', 'message': 'Data not found.'}
{'status': 'NOT_FOUND', 'request_id': '711849afafb4cae7bd76d36bd346a6de', 'message': 'Data n